# Sphere Fit

**Problem definition:**
We have $n$ 3D points which are spread along a spherical surface and we want to find to best sphere equation which describes them.

**Now in mathamatical notation:**
A sphere at point $(a,b,c)$ with radius $r$ is defined by the equation:

$\begin{equation}
(x-a)^2+(y-b)^2+(z-c)^2=r^2
\end{equation}$

We have $\{\mathbf{X}i\}^n_{i=1}$ points which we want to fit, so we will define the cost function as the distance of each point from the sphere surface:

$\begin{equation}
E(a,b,c,r)=\sum_i^n{\left(\sqrt{(x_i-a)^2+(y_i-b)^2+(z_i-c)^2}-r\right)^2}
\end{equation}$

Where we will denote $R(a,b,c)=\sqrt{(x_i-a)^2+(y_i-b)^2+(z_i-c)^2}$, such that $E(a,b,c,r)=\sum_i^n{\left(R^2-r\right)^2}$ 

In general, the solution to this problem is non-linear.

## Generate test data

We will generate some random points on a sphere.

In [16]:
import numpy as np
import scipy as sp
import scipy.optimize
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [25]:
def generate_pts_on_sphere(num_points, center, radius, noise_std=0):
    """ Generate points on a sphere """
    # ϕ∈[0,2π] is the azimuthal angle and θ∈[0,π] is the polar angle
    phi_vec = 2 * np.pi * np.random.uniform(low=0, high=1.0, size=num_points)
    theta_vec = np.arccos(1 - 2*np.random.uniform(low=0, high=1.0, size=num_points))
    x = np.sin(theta_vec) * np.cos(phi_vec)
    y = np.sin(theta_vec) * np.sin(phi_vec)
    z = np.cos(theta_vec)
    if noise_std > 0:
        noise = np.random.normal(loc=0, scale=noise_std, size=num_points)
        pts = center + (radius + noise[:, np.newaxis]) * np.vstack((x, y, z)).T
    else:
        pts = center + radius * np.vstack((x, y, z)).T
    return pts

center_gt = np.array([1, 1, 1])
radius_gt = 1
pts = generate_pts_on_sphere(num_points=100, center=center_gt, radius=radius_gt)

In [26]:
# Plot sphere wire-frame
def plot_sphere_surface(center, radius, **kwargs):
    theta_vec = np.linspace(0, np.pi, 20)
    phi_vec = np.linspace(0, 2*np.pi, 20)
    x_sphere = center[0]+ radius * np.outer(np.cos(phi_vec), np.sin(theta_vec))
    y_sphere = center[1]+ radius * np.outer(np.sin(phi_vec), np.sin(theta_vec))
    z_sphere = center[2]+ radius * np.outer(np.ones(np.size(phi_vec)), np.cos(theta_vec))
    sphere_trace= go.Surface(
        x=x_sphere,
        y=y_sphere,
        z=z_sphere,
        surfacecolor=np.zeros_like(z_sphere),
        opacity=0.6,
        showscale=False,
        **kwargs)
    return sphere_trace
sphere_gt_trace = plot_sphere_surface(center=center_gt, radius=radius_gt)

# Plot points
layout = go.Layout(autosize=True, height=400, title="3D Points",
                   margin=dict(l=0, r=0, b=0, t=0))
pts_trace = go.Scatter3d(x=pts[:, 0], y=pts[:, 1], z=pts[:, 2], mode='markers', marker={'size': 4, 'color':'blue'})
data = go.Data([pts_trace, sphere_gt_trace])
figure = go.Figure(data=data,layout=layout)
config = {'showLink': False}
iplot(figure, config=config)

## Linear Methods
Assuming the noise in our data is pretty small, we can start from the following linear approximation:

$\begin{equation}
E(a,b,c,r) = \sum^n_i{(R - r)^2} \approx \sum^n_i{\left(R^2 - r^2\right)}
\end{equation}$

After some algebra we get: $2x_ia + 2y_ib + 2z_ic + r² - a² - b² - c² = x_i² + y_i² + z_i²$

which can be written using matrix multiplication and can be solved using regular linear least-squares methods

$\begin{equation}
\begin{bmatrix}
2x_0 & 2y_0 & 2z_0 & 1 \\
2x_1 & 2y_1 & 2z_1 & 1 \\
\vdots&\vdots&\vdots&\vdots\\
2x_n & 2y_n & 2z_n & 1 \\
\end{bmatrix}
\cdot
\begin{bmatrix}
a\\b\\c\\r^2-a^2-b^2-c^2\\
\end{bmatrix}
=
\begin{bmatrix}
x_0^2+y_0^2+z_0^2 \\
x_1^2+y_1^2+z_1^2 \\
\vdots\\
x_n^2+y_n^2+z_n^2 \\
\end{bmatrix}
\end{equation}$

In [27]:
def sphere_fit_lsq(pts3d):
    A = np.hstack((2*pts3d, np.ones((len(pts3d), 1))))
    b = np.sum(pts3d**2, axis=1)

    # Solve linear system
    x, residules, rank, singval = np.linalg.lstsq(A, b, rcond=None)

    # Solve for the radius
    radius = np.sqrt(x[0]**2 + x[1]**2 + x[2]**2 + x[3])

    return radius, x[0], x[1], x[2]

The first run will be on perfect data, where we expect very accurate fit, up to negligble numeric errors.

In [28]:
r, a, b, c = sphere_fit_lsq(pts)
center_err = np.linalg.norm(center_gt - np.array([a, b, c]), ord=2)
residuals = np.linalg.norm(pts - np.array([a, b, c]), axis=1, ord=2) - r
msr = np.sqrt(np.mean((residuals)**2))
print("Estimated Sphere: (x-{:.3f})^2+(y-{:.3f})^2+(z-{:.3f})^2={:.3f}^2".format(a, b, c, r))
print("Distance from GT sphere center: {:.3f}".format(center_err))
print("Average Residuals: {:.3f}".format(msr))

Estimated Sphere: (x-1.000)^2+(y-1.000)^2+(z-1.000)^2=1.000^2
Distance from GT sphere center: 0.000
Average Residuals: 0.000


## Adding noise

We will now add Gaussian noise in the radial direction to each point and re-run the least-squares fit.

In [29]:
pts_noisy = generate_pts_on_sphere(num_points=1000, center=center_gt, radius=radius_gt, noise_std=0.2)

r, a, b, c = sphere_fit_lsq(pts_noisy)
center_err = np.linalg.norm(center_gt - np.array([a, b, c]), ord=2)
residuals = np.linalg.norm(pts_noisy - np.array([a, b, c]), axis=1, ord=2) - r
pts_err = np.linalg.norm(pts_noisy - center_gt, axis=1, ord=2) - radius_gt
msr = np.sqrt(np.mean((residuals)**2))
print("Estimated Sphere: (x-{:.3f})^2+(y-{:.3f})^2+(z-{:.3f})^2={:.3f}^2".format(a, b, c, r))
print("Distance from GT sphere center: {:.3f}".format(center_err))
print("Average Residuals: {:.3f}".format(msr))

Estimated Sphere: (x-1.007)^2+(y-0.990)^2+(z-0.989)^2=1.014^2
Distance from GT sphere center: 0.017
Average Residuals: 0.201


In [30]:
pts_trace = go.Scatter3d(x=pts_noisy[:, 0], y=pts_noisy[:, 1], z=pts_noisy[:, 2],
                         mode='markers',
                         marker={'size': 4, 'color': residuals, 'colorscale': 'Viridis', 'showscale':True,
                                 'colorbar':{'title':'Residuals'}})
sphere_trace = plot_sphere_surface(np.array([a, b, c]), r)
data = go.Data([pts_trace, sphere_trace])
figure = go.Figure(data=data,layout=layout)
iplot(figure, config=config)

We can see that for noisy data it's hard to get a good fit.
Before we advance to non-linear method, it's worth mentioning that we can improve the compute time by using another formulation to mimimze the linear approximation error term by implementing [Fast Geometric Fit Algorithm for Sphere Using Exact Solution, Sumith YD (2015)](https://arxiv.org/pdf/1506.02776.pdf)

In [31]:
def sphere_fit_lsq_fast(pts3d):
    """
    Fit a sphere to X,Y, and Z data points, using a closed form for the
    solution (opposed to using an array the size of the data set like in
    'sphere_fit_lsq').

    Minimizes Sum((x-xc)^2+(y-yc)^2+(z-zc)^2-r^2)^2
    x,y,z are the data, xc,yc,zc are the sphere's center, and r is the radius

    Reference:
     - Fast Geometric Fit Algorithm for Sphere Using Exact Solution, Sumith YD (2015)
       https://arxiv.org/pdf/1506.02776.pdf
    :param pts3d: NumPy array Nx3 of x,y,z
    :return: [r, xc, yc, zc] - radius and center point
    """
    N = len(pts3d)
    Sx, Sy, Sz = np.sum(pts3d, axis=0)
    Sxx = np.sum(pts3d[:, 0] * pts3d[:, 0])
    Syy = np.sum(pts3d[:, 1] * pts3d[:, 1])
    Szz = np.sum(pts3d[:, 2] * pts3d[:, 2])
    Sxy = np.sum(pts3d[:, 0] * pts3d[:, 1])
    Sxz = np.sum(pts3d[:, 0] * pts3d[:, 2])
    Syz = np.sum(pts3d[:, 1] * pts3d[:, 2])
    Sxxx = np.sum(pts3d[:, 0] * pts3d[:, 0] * pts3d[:, 0])
    Syyy = np.sum(pts3d[:, 1] * pts3d[:, 1] * pts3d[:, 1])
    Szzz = np.sum(pts3d[:, 2] * pts3d[:, 2] * pts3d[:, 2])
    Sxyy = np.sum(pts3d[:, 0] * pts3d[:, 1] * pts3d[:, 1])
    Sxzz = np.sum(pts3d[:, 0] * pts3d[:, 2] * pts3d[:, 2])
    Sxxy = np.sum(pts3d[:, 0] * pts3d[:, 0] * pts3d[:, 1])
    Sxxz = np.sum(pts3d[:, 0] * pts3d[:, 0] * pts3d[:, 2])
    Syyz = np.sum(pts3d[:, 1] * pts3d[:, 1] * pts3d[:, 2])
    Syzz = np.sum(pts3d[:, 1] * pts3d[:, 2] * pts3d[:, 2])

    A1 = Sxx + Syy + Szz

    a = 2 * Sx * Sx - 2 * N * Sxx
    b = 2 * Sx * Sy - 2 * N * Sxy
    c = 2 * Sx * Sz - 2 * N * Sxz
    d = -N * (Sxxx + Sxyy + Sxzz) + A1 * Sx

    e = 2 * Sx * Sy - 2 * N * Sxy
    f = 2 * Sy * Sy - 2 * N * Syy
    g = 2 * Sy * Sz - 2 * N * Syz
    h = -N * (Sxxy + Syyy + Syzz) + A1 * Sy

    j = 2 * Sx * Sz - 2 * N * Sxz
    k = 2 * Sy * Sz - 2 * N * Syz
    l = 2 * Sz * Sz - 2 * N * Szz
    m = -N * (Sxxz + Syyz + Szzz) + A1 * Sz

    delta = a * (f * l - g * k) - e * (b * l - c * k) + j * (b * g - c * f)

    xc = (d * (f * l - g * k) - h * (b * l - c * k) + m * (b * g - c * f)) / delta
    yc = (a * (h * l - m * g) - e * (d * l - m * c) + j * (d * g - h * c)) / delta
    zc = (a * (f * m - h * k) - e * (b * m - d * k) + j * (b * h - d * f)) / delta
    R = np.sqrt(xc**2 + yc**2 + zc**2 + (A1 -2 * (xc * Sx + yc * Sy + zc * Sz)) / N)
    return R, xc, yc, zc

In [32]:
r, a, b, c = sphere_fit_lsq_fast(pts_noisy)
center_err = np.linalg.norm(center_gt - np.array([a, b, c]), ord=2)
residuals = np.linalg.norm(pts_noisy - np.array([a, b, c]), axis=1, ord=2) - r
msr = np.sqrt(np.mean((residuals)**2))
print("Estimated Sphere: (x-{:.3f})^2+(y-{:.3f})^2+(z-{:.3f})^2={:.3f}^2".format(a, b, c, r))
print("Distance from GT sphere center: {:.3f}".format(center_err))
print("Average Residuals: {:.3f}".format(msr))

Estimated Sphere: (x-1.007)^2+(y-0.990)^2+(z-0.989)^2=1.014^2
Distance from GT sphere center: 0.017
Average Residuals: 0.201


We see we get the exact same result as the regular least-square.

### Runtime Speed Comparison

To compare the runtime we will generate much more points.

**Note:** Of course that comparing Python implementations which none of them was optimized is far from a fair compraison, but we just want to demonstrate the difference between the two.

In [33]:
# Comparing runtime
pts_tmp = generate_pts_on_sphere(num_points=100000, center=np.array([0, 0, 0]), radius=1)
%timeit sphere_fit_lsq(pts_tmp)
%timeit sphere_fit_lsq_fast(pts_tmp)

7.21 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.94 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


So we see that for 100,000 points we get a speed boost of ~85% which is nice for not a lot of effort.

## Non-linear Methods



In [34]:
def sphere_fit_fixed_iteration(pts3d, init_center, radius,
                               max_iterations=1000, convergance_th=1e-6, verbose=True):
    """

    :param pts3d:
    :param init_center:
    :param radius:
    :param max_iterations:
    :param convergance_th:
    :param verbose: boolean, print debug data.
    :return:
    """
    pts_3d_mean = np.mean(pts3d, axis=0)
    center = init_center.copy()
    prev_center = center.copy()
    for n_iter in range(max_iterations):
        Li = np.sqrt(np.sum((center - pts3d)**2, axis=1))
        center = pts_3d_mean + radius*np.mean((center - pts3d)/Li[:, np.newaxis], axis=0)
        delta = np.sqrt(np.mean((center - prev_center)**2))
        if verbose:
            print("Iteration #{} | delta={:.6f} | center=({:.5f},{:.5f},{:.5f})".format(
                n_iter, delta, center[0], center[1], center[2]))
        if delta < convergance_th:
            # Solution converged!
            break
        prev_center = center.copy()
    return radius, center[0], center[1], center[2]

In [35]:
r, a, b, c = sphere_fit_fixed_iteration(pts_noisy, init_center=center_gt, radius=radius_gt)
center_err = np.linalg.norm(center_gt - np.array([a, b, c]), ord=2)
residuals = np.linalg.norm(pts_noisy - center_gt, axis=1, ord=2) - radius_gt
rmse = np.sqrt(np.mean((residuals)**2))
print("Estimated Sphere: (x-{:.3f})^2+(y-{:.3f})^2+(z-{:.3f})^2={:.3f}^2".format(a, b, c, r))
print("Distance from GT sphere center: {:.3f}".format(center_err))
print("RMSE: {:.3f}".format(rmse))

Iteration #0 | delta=0.001816 | center=(1.00101,0.99956,0.99705)
Iteration #1 | delta=0.001262 | center=(1.00172,0.99926,0.99501)
Iteration #2 | delta=0.000877 | center=(1.00221,0.99906,0.99358)
Iteration #3 | delta=0.000610 | center=(1.00256,0.99892,0.99259)
Iteration #4 | delta=0.000424 | center=(1.00280,0.99882,0.99191)
Iteration #5 | delta=0.000295 | center=(1.00296,0.99875,0.99143)
Iteration #6 | delta=0.000205 | center=(1.00308,0.99871,0.99110)
Iteration #7 | delta=0.000143 | center=(1.00316,0.99868,0.99087)
Iteration #8 | delta=0.000099 | center=(1.00322,0.99865,0.99071)
Iteration #9 | delta=0.000069 | center=(1.00326,0.99864,0.99059)
Iteration #10 | delta=0.000048 | center=(1.00329,0.99863,0.99052)
Iteration #11 | delta=0.000033 | center=(1.00331,0.99862,0.99046)
Iteration #12 | delta=0.000023 | center=(1.00332,0.99862,0.99043)
Iteration #13 | delta=0.000016 | center=(1.00333,0.99861,0.99040)
Iteration #14 | delta=0.000011 | center=(1.00334,0.99861,0.99038)
Iteration #15 | delt

In [36]:
def _sphere_cost_func(x, pts3d, radius):
    """
    Function which computes the vector of residuals for non-linear
    least-squares.
    p = [a,b,c] - a,b,c are center coords to be fit
    distance = sqrt( (x-a)**2 + (y-b)**2 + (z-c)**2 )
    err = distance - r
    err is distance from input point to current fitted surface
    :return:
    """
    r_est = np.sqrt(np.sum((pts3d - x)**2, axis=1))
    err = np.sqrt(np.mean((r_est - radius)**2))
    # err = (r_est - radius)**2
    return err


def _sphere_cost_jacobian(x, pts3d, radius):
    """
    Jacobian for the non-linear least squares
    """
    tmp = x - pts3d
    t1 = np.mean(tmp, axis=0)
    t2 = -radius*np.mean(tmp/np.sqrt(np.sum(tmp**2, axis=1))[:, np.newaxis], axis=0)
    # t2 = -radius*tmp/np.sqrt(np.sum(tmp**2, axis=1))[:, np.newaxis]
    jac = 2*(t1 + t2)
    return jac


def sphere_fit_non_linear_lsq(pts3d, init_center, radius):
    """
    Fit a sphere center to X,Y, and Z data points, when the radius is known.

    :param pts3d: NumPy array Nx3 of x,y,z
    :return: [r, a, b, c] - radius and center points of the best fit sphere
    """
    res = sp.optimize.least_squares(_sphere_cost_func,
                                    x0=init_center,
                                    jac=_sphere_cost_jacobian,
                                    max_nfev=10000,
                                    xtol=1e-10,
                                    ftol=1e-12,
                                    gtol=1e-12,
                                    args=(pts3d, radius),
                                    method='dogbox',
                                    verbose=1)
    return radius, res.x[0], res.x[1], res.x[2]

In [37]:
r, a, b, c = sphere_fit_non_linear_lsq(pts3d=pts_noisy, init_center=center_gt, radius=radius_gt)
center_err = np.linalg.norm(center_gt - np.array([a, b, c]), ord=2)
residuals = np.linalg.norm(pts_noisy - center_gt, axis=1, ord=2) - radius_gt
rmse = np.sqrt(np.mean((residuals)**2))
print("Estimated Sphere: (x-{:.3f})^2+(y-{:.3f})^2+(z-{:.3f})^2={:.3f}^2".format(a, b, c, r))
print("Distance from GT sphere center: {:.3f}".format(center_err))
print("RMSE: {:.3f}".format(rmse))

`ftol` termination condition is satisfied.
Function evaluations 23, initial cost 2.0005e-02, final cost 1.9989e-02, first-order optimality 6.21e-09.
Estimated Sphere: (x-1.003)^2+(y-0.999)^2+(z-0.990)^2=1.000^2
Distance from GT sphere center: 0.010
RMSE: 0.200


**TODO:**
- Evaluate different sector angles, number of points and noise to show performance difference
- Change to Scipy minimize to show difference with and without Jacobian and Hessian 

# References

**Linear methods:**
- [Fast Geometric Fit Algorithm for Sphere Using Exact Solution](https://arxiv.org/abs/1506.02776)
- [Least-squares best-fit geometric elements, Forbes (1989)](https://www.researchgate.net/publication/274371520_Least-squares_best-fit_geometric_elements)

**Nonlinear Methods:**
- [Least Squares Fitting of Data, David Eberly (Geometric Tools)](https://www.geometrictools.com/Documentation/LeastSquaresFitting.pdf).
- [Geometric least-squares fitting of spheres, cylinders, cones and tori](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.104.3756&rep=rep1&type=pdf)
- [Least-squares orthogonal distances fitting of circle, sphere, ellipse, hyperbola, and parabola](http://oana7cw0r.bkt.clouddn.com/GeometricEllipseFitting/1-s2.0-S0031320300001527-main.pdf)
- Non-linear Least-Squares II: Circle, Sphere, and Cylinder, chapter 17 (https://doi.org/10.1007/978-1-84800-297-5_17)